# Settings

In [4]:
import json
import requests
import pandas as pd

with open('api_key.txt', encoding='utf8') as f:
    API_KEY = f.readline()

# display max numnber of rows
pd.set_option('display.max_rows', None)

# display max numnber of columns
pd.set_option('display.max_columns', None)

# reset options
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

# Get Data via API

In [6]:
# make API request to get all possible endpoints for all available metrics
res = requests.get('https://api.glassnode.com/v2/metrics/endpoints',
    params={'api_key': API_KEY})

# convert to pandas dataframe
df = pd.read_json(res.text)

In [51]:
# filter on tier 1 metrics
df_fil = df[df["tier"] == 1]
df_fil.tier.value_counts()

1    75
Name: tier, dtype: int64

In [68]:
import re

def get_att_name(path):
    # capture path name afer v1/metrics
    s = re.match("^/v1/metrics/(.*)", path)
    att = s.groups()[0]
    # replace slashes with underscores
    att_name = att.replace("/", "_")
    return att_name

def get_data(df_fil):
    for count, path in enumerate(df_fil["path"]):
        att_name = get_att_name(path)
        try:
            res = requests.get(f'https://api.glassnode.com{path}',
                               params={'a': 'ETH','api_key': API_KEY})
            # convert to pandas dataframe
            df = pd.read_json(res.text, convert_dates=['t'])
            df.rename(columns={"t": "date", "v": att_name}, inplace=True)
            if count == 0:
                df_final = df.copy()
            else:
                df_final = pd.merge(df_final, df, how="outer", on="date")
        # metrics which are not available for ETH
        except Exception as e:
            pass
#             if hasattr(e, "message"):
#                 print(att_name, e.message)
#             else:
#                 print(att_name, e)
    df_final.dropna(inplace=True)
    df_final.to_csv("eth_data.csv", index=False)
    return df_final

In [69]:
# get data
df_final = get_data(df_fil)
df_final.head(10)

,date,addresses_count,addresses_sending_count,addresses_receiving_count,addresses_active_count,addresses_new_non_zero_count,mining_difficulty_mean,mining_difficulty_latest,mining_hash_rate_mean,transactions_count,transactions_rate,transactions_size_mean,transactions_size_sum,transactions_transfers_count,transactions_transfers_rate,transactions_transfers_volume_sum,transactions_transfers_volume_mean,transactions_transfers_volume_median,fees_gas_used_sum,fees_gas_used_mean,fees_gas_used_median,fees_gas_limit_tx_mean,fees_gas_limit_tx_median,fees_gas_price_mean,fees_gas_price_median,fees_volume_sum,fees_volume_mean,indicators_sopr
8,2015-08-07,10288,778,606,1085,500,1.470839e+12,1531935261642,8.827049e+10,2035.0,0.023553,0.0,0.0,1981.0,0.022928,2.008619e+06,1013.941711,18.977734,49353826.0,24075.037073,21000.0,45679.056098,22000.0,6.046842e+11,6.028009e+10,37.318414,0.018204,1.000000
9,2015-08-08,10641,582,439,800,353,1.586124e+12,1606016380751,9.644985e+10,2344.0,0.027130,0.0,0.0,1553.0,0.017975,1.358297e+07,8746.277229,10.000000,376006093.0,130512.354391,21000.0,245515.518917,90000.0,3.227136e+11,5.919973e+10,68.099970,0.023638,0.459823
10,2015-08-09,10894,457,489,731,253,1.709480e+12,1741399466631,1.013969e+11,1312.0,0.015185,0.0,0.0,1244.0,0.014398,1.165448e+06,936.855365,12.250237,38863003.0,29242.289691,21000.0,80584.452220,22000.0,4.754671e+11,5.800373e+10,14.098950,0.010609,0.403942
11,2015-08-10,11543,804,749,997,649,1.837696e+12,1948102094734,1.116431e+11,2027.0,0.023461,0.0,0.0,1525.0,0.017650,1.670884e+06,1095.661579,4.000000,74070061.0,36362.327442,21000.0,82231.792342,50000.0,4.216549e+11,5.570179e+10,31.165144,0.015300,0.360587
12,2015-08-11,13432,2119,2106,2339,1889,2.036391e+12,2171897473895,1.240757e+11,4955.0,0.057350,0.0,0.0,3286.0,0.038032,1.485697e+06,452.129195,0.119664,163481740.0,32940.104775,21000.0,74161.196051,22000.0,7.783882e+10,5.000000e+10,11.311449,0.002279,0.614763
13,2015-08-12,13744,565,673,904,312,2.207080e+12,2248237602936,1.308930e+11,2009.0,0.023252,0.0,0.0,1811.0,0.020961,7.474474e+05,412.726366,10.000000,70102332.0,34431.400786,21000.0,153478.538310,30000.0,4.449024e+11,5.794943e+10,32.586992,0.016005,0.609176
14,2015-08-13,14274,858,989,1273,530,2.336980e+12,2419921828013,1.428465e+11,2834.0,0.032801,0.0,0.0,2604.0,0.030139,1.787811e+06,686.563308,10.000000,88234087.0,31046.476777,21000.0,88564.727657,22000.0,2.686835e+11,5.740754e+10,24.929169,0.008772,0.839077
15,2015-08-14,14810,1098,1172,1592,536,2.671253e+12,3177173581368,1.771801e+11,3166.0,0.036644,0.0,0.0,3061.0,0.035428,2.295429e+06,749.895158,12.244024,78746522.0,24809.868305,21000.0,66965.374606,22000.0,1.934555e+11,5.741340e+10,14.354834,0.004523,0.862419
16,2015-08-15,15175,845,1050,1309,365,3.378028e+12,3636757923666,2.102737e+11,2269.0,0.026262,0.0,0.0,2179.0,0.025220,4.631229e+06,2125.391905,11.177469,59565914.0,26079.647110,21000.0,58293.196585,22000.0,1.443689e+11,5.751020e+10,8.836593,0.003869,1.070524
17,2015-08-16,15450,748,976,1202,275,3.631632e+12,3884083612249,2.149781e+11,2438.0,0.028218,0.0,0.0,2353.0,0.027234,1.708589e+06,726.132127,8.315054,58241191.0,23869.340574,21000.0,63010.490164,22000.0,1.209401e+11,5.737540e+10,6.677238,0.002737,0.696735


In [71]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2108 entries, 8 to 2115
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date                                  2108 non-null   datetime64[ns]
 1   addresses_count                       2108 non-null   int64         
 2   addresses_sending_count               2108 non-null   int64         
 3   addresses_receiving_count             2108 non-null   int64         
 4   addresses_active_count                2108 non-null   int64         
 5   addresses_new_non_zero_count          2108 non-null   int64         
 6   mining_difficulty_mean                2108 non-null   float64       
 7   mining_difficulty_latest              2108 non-null   int64         
 8   mining_hash_rate_mean                 2108 non-null   float64       
 9   transactions_count                    2108 non-null   float64       
 10  

In [72]:
df_final.describe()

,addresses_count,addresses_sending_count,addresses_receiving_count,addresses_active_count,addresses_new_non_zero_count,mining_difficulty_mean,mining_difficulty_latest,mining_hash_rate_mean,transactions_count,transactions_rate,transactions_size_mean,transactions_size_sum,transactions_transfers_count,transactions_transfers_rate,transactions_transfers_volume_sum,transactions_transfers_volume_mean,transactions_transfers_volume_median,fees_gas_used_sum,fees_gas_used_mean,fees_gas_used_median,fees_gas_limit_tx_mean,fees_gas_limit_tx_median,fees_gas_price_mean,fees_gas_price_median,fees_volume_sum,fees_volume_mean,indicators_sopr
count,2.108000e+03,2108.000000,2108.000000,2108.000000,2108.000000,2.108000e+03,2.108000e+03,2.108000e+03,2.108000e+03,2108.000000,2108.0,2108.0,2.108000e+03,2108.000000,2.108000e+03,2108.000000,2108.000000,2.108000e+03,2108.000000,2108.000000,2108.000000,2108.000000,2.108000e+03,2.108000e+03,2108.000000,2108.000000,2108.000000
mean,3.608834e+07,96424.099146,138105.467268,218137.833017,55184.925047,1.937266e+15,1.948696e+15,1.355472e+14,5.213232e+05,6.033834,0.0,0.0,2.506444e+05,2.900976,3.977585e+06,42.243397,0.566588,3.351110e+10,54623.657171,27048.507116,166189.536813,76394.999051,4.159939e+10,3.267841e+10,1738.639381,0.002128,0.996462
std,3.452723e+07,81041.125077,108672.027306,171333.304468,51658.164021,1.583882e+15,1.596792e+15,1.164163e+14,4.033135e+05,4.667980,0.0,0.0,1.911830e+05,2.212767,5.019099e+06,208.127549,0.960252,2.733464e+10,16951.427410,7536.332935,55555.926913,18549.687761,5.848241e+10,4.256520e+10,3869.932564,0.003079,0.129518
min,1.028800e+04,457.000000,439.000000,731.000000,44.000000,1.470839e+12,1.531935e+12,8.827049e+10,1.312000e+03,0.015185,0.0,0.0,1.244000e+03,0.014398,6.860737e+04,2.375944,0.000000,3.886300e+07,22032.696751,21000.000000,45679.056098,22000.000000,7.292978e+09,1.200000e+09,5.507452,0.000373,0.360587
25%,7.612025e+05,9800.250000,18818.500000,20783.000000,2764.250000,1.040546e+14,1.049934e+14,7.291593e+12,4.973275e+04,0.575611,0.0,0.0,4.301525e+04,0.497862,1.301208e+06,6.067478,0.100289,2.030627e+09,38935.664982,21000.000000,131827.820713,65000.000000,1.489395e+10,1.000000e+10,54.923599,0.000687,0.961995
50%,3.399438e+07,92238.500000,140424.000000,230047.500000,48254.000000,2.202665e+15,2.205370e+15,1.530092e+14,5.597435e+05,6.478513,0.0,0.0,2.510800e+05,2.906019,2.145330e+06,9.995002,0.174072,3.774507e+10,56864.558224,22392.000000,158545.047374,80000.000000,2.280904e+10,2.000000e+10,391.475712,0.000890,1.006938
75%,6.260956e+07,139538.250000,202224.000000,331295.500000,83019.750000,2.975433e+15,2.999744e+15,2.054996e+14,7.771410e+05,8.994687,0.0,0.0,3.641578e+05,4.214789,3.683534e+06,42.839041,1.002248,4.744492e+10,67175.989488,34105.000000,198694.280800,90000.000000,5.074991e+10,4.400000e+10,753.429050,0.001925,1.037700
max,1.163396e+08,444027.000000,595595.000000,794922.000000,348434.000000,7.886198e+15,8.215645e+15,5.908259e+14,1.632453e+06,18.894132,0.0,0.0,1.130165e+06,13.080613,4.648852e+07,8746.277229,18.977734,9.644762e+10,206886.047017,87179.000000,890603.044116,300000.000000,9.395646e+11,5.440000e+11,42763.252558,0.032143,2.391537
